In [1]:
import pycocotools
import os
import numpy as np
import torch
import torch.utils.data
import cv2
import pandas as pd 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import nn
import torchvision.models as models
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [2]:
from python.engine import train_one_epoch, evaluate
from python import utils
import python.transforms as T

In [3]:
def parse_one_annot(path):
    data = {}
    for i,e in enumerate(os.listdir(path)):
        data_bach = pd.read_csv(os.path.join(path,e))
        data_bach = data_bach.values[0]
        x1 = int((data_bach[0].split(" ")[0]).split(".")[0])
        y1 = int((data_bach[0].split(" ")[1]).split(".")[0])
        x2 = int((data_bach[0].split(" ")[2]).split(".")[0])
        y2 = int((data_bach[0].split(" ")[3]).split(".")[0])
        data[e] = [x1,y1,x2,y2]
    return data

In [4]:
#a = parse_one_annot("Label")
#a['0.csv']

In [5]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, image, label, transforms=None):
        self.image = image
        self.transforms = transforms
        self.imgs = sorted(os.listdir(image))
        self.label =  parse_one_annot(label)
    
    def __getitem__(self, idx):
            # load images and bounding boxes
        img_path = os.path.join(self.image, self.imgs[idx])
        img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)
        box_list = self.label[self.imgs[idx].split(".")[0]+".csv"]
        box_list = np.expand_dims(box_list, axis=0)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
            # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
            # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        #target["image_id"] = image_id
        #target["area"] = area
        #target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
        
    def __len__(self):
        return len(self.imgs)

In [6]:
dataset = Dataset("Image","Label")
dataset.__getitem__(0)[0].shape

(745, 500, 3)

In [7]:
def get_model(num_classes):
        # load an object detection model pre-trained on COCO
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)# get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

In [8]:
def get_transform(train):
    transforms = []
        # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [9]:
# use our dataset and defined transformationsdataset
dataset = Dataset("Image","Label",get_transform(train=True))
dataset_test = Dataset("Image","Label",get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
a = int(len(dataset)*4/10)
print(a)
b =int(len(dataset)/2 - a)
print(b)
dataset = torch.utils.data.Subset(dataset, indices[:a])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-b:])                       

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,num_workers=0,collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0,collate_fn=utils.collate_fn)

print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

3372
843
We have: 8431 examples, 3372 are training and 843 testing


In [10]:
dataset.__getitem__(0)[0].shape

torch.Size([3, 594, 386])

In [11]:
torch.cuda.is_available()

True

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - raccoon and not racoon
num_classes = 2
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epoch
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [13]:
# let's train it for 10 epochs
num_epochs = 1
for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch,print_freq=10)# update the learning rate
    lr_scheduler.step()
        # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

d:\python\python38\lib\site-packages\torchvision\ops\boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Epoch: [0]  [   0/3372]  eta: 1:22:47  lr: 0.000010  loss: 0.7551 (0.7551)  loss_classifier: 0.6836 (0.6836)  loss_box_reg: 0.0164 (0.0164)  loss_objectness: 0.0386 (0.0386)  loss_rpn_box_reg: 0.0166 (0.0166)  time: 1.4731  data: 0.0110  max mem: 1316
Epoch: [0]  [  10/3372]  eta: 0:28:54  lr: 0.000060  loss: 0.7598 (0.7691)  loss_classifier: 0.6285 (0.6105)  loss_box_reg: 0.0903 (0.0817)  loss_objectness: 0.0149 (0.0639)  loss_rpn_box_reg: 0.0144 (0.0130)  time: 0.5159  data: 0.0051  max mem: 1615
Epoch: [0]  [  20/3372]  eta: 0:26:16  lr: 0.000110  loss: 0.5504 (0.5887)  loss_classifier: 0.4164 (0.4447)  loss_box_reg: 0.0903 (0.0791)  loss_objectness: 0.0101 (0.0504)  loss_rpn_box_reg: 0.0105 (0.0145)  time: 0.4203  data: 0.0049  max mem: 1615
Epoch: [0]  [  30/3372]  eta: 0:25:21  lr: 0.000160  loss: 0.3111 (0.4905)  loss_classifier: 0.1628 (0.3437)  loss_box_reg: 0.0921 (0.0837)  loss_objectness: 0.0101 (0.0485)  loss_rpn_box_reg: 0.0116 (0.0145)  time: 0.4221  data: 0.0075  max me

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "model")

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
import torchvision.models as models
import time

def get_model(num_classes):
    # load an object detection model pre-trained on COCO
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)# get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

print(torch.cuda.is_available())
loaded_model = get_model(num_classes = 2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loaded_model.to(device)
loaded_model.load_state_dict(torch.load("model"))

In [ ]:
image = cv2.imread("Image/46.jpg")
img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
img = torch.tensor(img.transpose(2,1,0)).cuda()

#put the model in evaluation mode
loaded_model.eval()

with torch.no_grad():
    a = time.time()
    prediction = loaded_model([img])
    b = time.time()
    print(b-a)
    
for element in range(len(prediction[0]["boxes"])):
    boxes = prediction[0]["boxes"][element].cpu().numpy()
    score = np.round(prediction[0]["scores"][element].cpu().numpy(),decimals= 4)
    cv2.rectangle(image, (int(boxes[1]), int(boxes[0])), (int(boxes[3]), int(boxes[2])), (0, 255, 0), 2, cv2.LINE_AA)
    break
         
plt.figure()        
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))